###### Load Packages

In [88]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation, metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
#from sklearn.grid_search import GridSearchCV

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

###### Load Data

In [89]:
%%time
train_df = pd.read_csv( "train.csv" )
test_df = pd.read_csv( "test.csv" )

MemoryError: 

In [90]:
# To verify the data types and data format
train_df.info()
train_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


(200000, 202)

In [91]:
test_df.info()
test_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 201 entries, ID_code to var_199
dtypes: float64(200), object(1)
memory usage: 306.7+ MB


(200000, 201)

In [92]:
train_df.head( 10 )

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
5,train_5,0,11.4763,-2.3182,12.6080,8.6264,10.9621,3.5609,4.5322,15.2255,...,-6.3068,6.6025,5.2912,0.4403,14.9452,1.0314,-3.6241,9.7670,12.5809,-4.7602
6,train_6,0,11.8091,-0.0832,9.3494,4.2916,11.1355,-8.0198,6.1961,12.0771,...,8.7830,6.4521,3.5325,0.1777,18.3314,0.5845,9.1104,9.1143,10.8869,-3.2097
7,train_7,0,13.5580,-7.9881,13.8776,7.5985,8.6543,0.8310,5.6890,22.3262,...,13.1700,6.5491,3.9906,5.8061,23.1407,-0.3776,4.2178,9.4237,8.6624,3.4806
8,train_8,0,16.1071,2.4426,13.9307,5.6327,8.8014,6.1630,4.4514,10.1854,...,1.4298,14.7510,1.6395,1.4181,14.8370,-1.9940,-1.0733,8.1975,19.5114,4.8453
9,train_9,0,12.5088,1.9743,8.8960,5.4508,13.6043,-16.2859,6.0637,16.8410,...,0.5543,6.3160,1.0371,3.6885,14.8344,0.4467,14.1287,7.9133,16.2375,14.2514


In [93]:
test_df.head(10)

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846
5,test_5,5.9862,-2.2913,8.6058,7.0685,14.2465,-8.6761,4.2467,14.7632,1.8790,...,-2.1115,7.1178,-0.4249,8.8781,14.9438,-2.2151,-6.0233,9.8117,17.1127,10.8240
6,test_6,8.4624,-6.1065,7.3603,8.2627,12.0104,-7.2073,4.1670,13.0809,-4.3004,...,12.3609,6.8661,4.0971,8.8484,17.5010,0.0295,7.7443,9.1509,18.4736,5.1499
7,test_7,17.3035,-2.4212,13.3989,8.3998,11.0777,9.6449,5.9596,17.8477,-4.8068,...,4.4676,4.4214,0.9303,1.4994,15.2648,-1.7931,6.5316,10.4855,23.4631,0.7283
8,test_8,6.9856,0.8402,13.7161,4.7749,8.6784,-13.7607,4.3386,14.5843,2.5883,...,-3.4657,7.8754,2.4698,-0.0362,16.7144,0.1221,-1.4328,9.9207,16.9865,-3.3304
9,test_9,10.3811,-6.9348,14.6690,9.0941,11.9058,-10.8018,3.4508,20.2816,-1.4112,...,1.8052,11.0723,0.8907,4.7680,15.1425,0.6075,-4.4447,9.5788,15.8146,9.3457


In [94]:
TrainMissingCount = train_df.isnull().sum()
print( TrainMissingCount )

ID_code    0
target     0
var_0      0
var_1      0
var_2      0
var_3      0
var_4      0
var_5      0
var_6      0
var_7      0
var_8      0
var_9      0
var_10     0
var_11     0
var_12     0
var_13     0
var_14     0
var_15     0
var_16     0
var_17     0
var_18     0
var_19     0
var_20     0
var_21     0
var_22     0
var_23     0
var_24     0
var_25     0
var_26     0
var_27     0
          ..
var_170    0
var_171    0
var_172    0
var_173    0
var_174    0
var_175    0
var_176    0
var_177    0
var_178    0
var_179    0
var_180    0
var_181    0
var_182    0
var_183    0
var_184    0
var_185    0
var_186    0
var_187    0
var_188    0
var_189    0
var_190    0
var_191    0
var_192    0
var_193    0
var_194    0
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 202, dtype: int64


In [95]:
TestMissingCount = test_df.isnull().sum()
print( TestMissingCount )

ID_code    0
var_0      0
var_1      0
var_2      0
var_3      0
var_4      0
var_5      0
var_6      0
var_7      0
var_8      0
var_9      0
var_10     0
var_11     0
var_12     0
var_13     0
var_14     0
var_15     0
var_16     0
var_17     0
var_18     0
var_19     0
var_20     0
var_21     0
var_22     0
var_23     0
var_24     0
var_25     0
var_26     0
var_27     0
var_28     0
          ..
var_170    0
var_171    0
var_172    0
var_173    0
var_174    0
var_175    0
var_176    0
var_177    0
var_178    0
var_179    0
var_180    0
var_181    0
var_182    0
var_183    0
var_184    0
var_185    0
var_186    0
var_187    0
var_188    0
var_189    0
var_190    0
var_191    0
var_192    0
var_193    0
var_194    0
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 201, dtype: int64


In [96]:
train_df.describe()

KeyError: MemoryError()

In [ ]:
test_df.describe()

Looks like, the target variable is train_df['target'] and the feature variables are from var_0 to var_199( 200 variables ). 
200000 entries are there in both train and test data set. Variables data types are same - float64.
There is no categorical variable in the dataset. So no encoding is required.
And no missing data in the training  and test data set. The variable count is same in both train and test data set.

As per the describe() function, the data looks good. no scaling is required. There is a difference between mean and standard deviation. But we can see the same variation between train and test data set.

In [97]:
def ValidateData( data ):
    '''
    Validation function to check the missing value, missing data percentage,
                                     minimum value, maximum value, standard deviation of the data
    '''
    missingCount = data.isnull().sum()
    percent = ( data.isnull().sum() / data.isnull().count() * 100 )
    dataStatus = pd.concat([ missingCount, percent ], axis=1, keys=['Missing Count', 'Missing Data Percentage'])
    '''
    minVal = []
    maxVal = []
    stdVal = []
    avgVal = []
    
    for col in data.columns:
        minVal.append( data[ col ].min())
        maxVal.append( data[ col ].max())
        avgVal.append( data[ col ].mean())
        stdVal.append( data[ col ].std())

    dataStatus[ 'min' ]  = minVal
    dataStatus[ 'max' ]  = maxVal
    dataStatus[ 'mean' ] = avgVal
    dataStatus[ 'std' ]  = stdVal
    '''
    return( np.transpose( dataStatus ))

In [98]:
%%time
ValidateData( train_df[list(train_df.columns[2:])] )

MemoryError: 

In [ ]:
- 

Wall time: 0 ns


In [28]:
X_train = train_df[list(train_df.columns[2:])]

In [29]:
X_train.head(10)

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
5,11.4763,-2.3182,12.6080,8.6264,10.9621,3.5609,4.5322,15.2255,3.5855,5.9790,...,-6.3068,6.6025,5.2912,0.4403,14.9452,1.0314,-3.6241,9.7670,12.5809,-4.7602
6,11.8091,-0.0832,9.3494,4.2916,11.1355,-8.0198,6.1961,12.0771,-4.3781,7.9232,...,8.7830,6.4521,3.5325,0.1777,18.3314,0.5845,9.1104,9.1143,10.8869,-3.2097
7,13.5580,-7.9881,13.8776,7.5985,8.6543,0.8310,5.6890,22.3262,5.0647,7.1971,...,13.1700,6.5491,3.9906,5.8061,23.1407,-0.3776,4.2178,9.4237,8.6624,3.4806
8,16.1071,2.4426,13.9307,5.6327,8.8014,6.1630,4.4514,10.1854,-3.1882,9.0827,...,1.4298,14.7510,1.6395,1.4181,14.8370,-1.9940,-1.0733,8.1975,19.5114,4.8453
9,12.5088,1.9743,8.8960,5.4508,13.6043,-16.2859,6.0637,16.8410,0.1287,7.9682,...,0.5543,6.3160,1.0371,3.6885,14.8344,0.4467,14.1287,7.9133,16.2375,14.2514


In [ ]:
y_train = train_df['target']

In [ ]:
y_train.head()

In [ ]:
X_test = test_df[list(test_df.columns[1:])]
X_test.head()

In [ ]:
y_test = pd.read_csv( "sample_submission.csv" )
y_test = train_df[list(y_test.columns[1:])]
y_test.head()

In [69]:
learning_rates = [0.01,0.05,0.075, 0.1,0.2,0.25,0.4,0.5]
for learning_rate in learning_rates:
    cv = GradientBoostingClassifier(n_estimators=10, learning_rate = learning_rate, max_features=60,
                                    min_samples_split =5,min_samples_leaf =15,
                                    max_depth = 5, random_state = 10)
    cv.fit(X_train, y_train)
    y_pred = cv.predict( X_test )
    fpr_gb, tpr_gb, _ = roc_curve( y_test, y_pred )
    roc_auc_gb = auc( fpr_gb, tpr_gb)
    print("Learning rate: {0:.3f}, Accuracy score (test): {1:.6f}, Area under ROC curve = {2:0.4f}".format( learning_rate, cv.score( X_test, y_test ), roc_auc_gb ))
#    print()
#     print("Confusion Matrix:")
#     print(confusion_matrix( y_test, y_pred ))
#     print()
#     print("Classification Report")
#     print(classification_report( y_test, y_pred ))
#     print()

#     print("Area under ROC curve = {:0.4f}".format(roc_auc_gb))

Learning rate: 0.010, Accuracy score (test): 0.899510, Area under ROC curve = 0.5000
Learning rate: 0.050, Accuracy score (test): 0.899510, Area under ROC curve = 0.5000
Learning rate: 0.075, Accuracy score (test): 0.899505, Area under ROC curve = 0.5000
Learning rate: 0.100, Accuracy score (test): 0.899500, Area under ROC curve = 0.5000
Learning rate: 0.200, Accuracy score (test): 0.898980, Area under ROC curve = 0.5001
Learning rate: 0.250, Accuracy score (test): 0.897950, Area under ROC curve = 0.4998
Learning rate: 0.400, Accuracy score (test): 0.893280, Area under ROC curve = 0.4997
Learning rate: 0.500, Accuracy score (test): 0.889960, Area under ROC curve = 0.4995


In [15]:
learningRate =0.05
xgcl = xgb.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = learningRate,
                max_depth = 7, alpha = 10, n_estimators = 300, silent=True, gamma=0, max_delta_step=0,
                reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=1,
                           seed=1,
                           missing=None)
xgcl.fit(X_train,y_train)

preds = xgcl.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
fpr_gb, tpr_gb, _ = roc_curve( y_test, preds )
roc_auc_gb = auc( fpr_gb, tpr_gb)
print("Learning rate: {0:.3f}, Accuracy score (test): {1:.6f}, Area under ROC curve = {2:0.4f}, rmse = {3:0.4f}".format( learningRate, xgcl.score( X_test, y_test ), roc_auc_gb , rmse))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Learning rate: 0.050, Accuracy score (test): 0.891215, Area under ROC curve = 0.5001, rmse = 0.3298


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
                            
 param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.4,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': 1
}

param = {
param['booster'] = 'gbtree'
param['objective'] = 'binary:logistic'
param['learning_rate'] = 0.02
param['base_score'] = 0.5
param['max_depth'] = 2
param['alpha'] = 10
param['n_estimators'] = 500
param['seed'] = 0
param['silent'] = 1
param["eval_metric"] = "error"
param['eta'] = 0.3
param['gamma'] = 0
param['min_child_weight']=1
param['max_delta_step'] = 0
param['subsample']= 1
param['colsample_bytree']=1
param['reg_lambda']= 0
param['scale_pos_weight']= 1
param['missing']= None
param['min_child_weight']= 1
param['verbosity']= 1

In [54]:
param = {
'booster': 'gbtree',
'base_score': 0.5,
'alpha': 10,
'n_estimators': 500,
'seed': 0,
'silent': 1,
'eta': 0.3,
'gamma': 0,
'min_child_weight': 1,
'max_delta_step': 0,
'subsample': 1,
'colsample_bytree': 1,
'reg_lambda': 0,
'scale_pos_weight': 1,
'missing': None,
'verbosity': 0,
'nthread ': 8,
}

In [63]:
learningRate = 0.05
eval_set = [(X_test, y_test)]
model = xgb.XGBClassifier(  max_depth = 3,
                            objective = 'binary:logistic',
                            booster= 'gbtree',
                            learning_rate = learningRate,
                            base_score= 0.5,
                            alpha= 10,
                            n_estimators= 500,
                            seed= 0,
                            silent= 1,
                            eta= 0.3,
                            gamma= 0,
                            min_child_weight= 1,
                            max_delta_step= 0,
                            subsample= 1,
                            colsample_bytree= 1,
                            reg_lambda= 0,
                            scale_pos_weight= 1,
                            missing= None,
                            verbosity= 1,
                            nthread = 8)
model.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="logloss", eval_set=eval_set, verbose=True)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.4f%%" % (accuracy * 100.0))
preds = y_pred;
rmse = np.sqrt(mean_squared_error(y_test, preds))
fpr_gb, tpr_gb, _ = roc_curve( y_test, preds )
roc_auc_gb = auc( fpr_gb, tpr_gb)
print("Learning rate: {0:.3f}, Accuracy score (test): {1:.6f}, Area under ROC curve = {2:0.4f}, rmse = {3:0.4f}".format( learningRate, model.score( X_test, y_test ), roc_auc_gb , rmse))

[0]	validation_0-logloss:0.662022
Will train until validation_0-logloss hasn't improved in 20 rounds.
[1]	validation_0-logloss:0.63391
[2]	validation_0-logloss:0.608404
[3]	validation_0-logloss:0.585249
[4]	validation_0-logloss:0.564147
[5]	validation_0-logloss:0.544895
[6]	validation_0-logloss:0.527283
[7]	validation_0-logloss:0.511155
[8]	validation_0-logloss:0.496356
[9]	validation_0-logloss:0.482811
[10]	validation_0-logloss:0.470336
[11]	validation_0-logloss:0.458876
[12]	validation_0-logloss:0.44834
[13]	validation_0-logloss:0.438638
[14]	validation_0-logloss:0.42969
[15]	validation_0-logloss:0.421481
[16]	validation_0-logloss:0.413905
[17]	validation_0-logloss:0.406943
[18]	validation_0-logloss:0.400492
[19]	validation_0-logloss:0.394577
[20]	validation_0-logloss:0.389131
[21]	validation_0-logloss:0.384074
[22]	validation_0-logloss:0.379447
[23]	validation_0-logloss:0.375203
[24]	validation_0-logloss:0.371276
[25]	validation_0-logloss:0.36768
[26]	validation_0-logloss:0.364328
[

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy: 89.9500%


NameError: name 'xgcl' is not defined

In [64]:
oof = np.zeros(len(train_df))

In [65]:
oof.shape

(200000,)

In [66]:
predictions = np.zeros(len(test_df))
predictions.shape

(200000,)

In [79]:
idx = 1;
predictions = np.zeros( len( test_df ))
oof         = np.zeros( len( train_df ))
eval_set = [(X_train, y_train)]
kf = StratifiedKFold( n_splits = 10, random_state = 44000, shuffle = False )
for train_index, val_index in kf.split( X_train.values, y_train.values ):
    print('\n{} of kfold {}'.format( idx, kf.n_splits ))
    xtr, xvl = X_train.loc[ train_index ], X_train.loc[ train_index ]
    ytr, yvl = y_test.loc[ val_index ], y_test.loc[ val_index ]
    xgbModel = xgb.XGBClassifier(  max_depth = 3,
                            objective = 'binary:logistic',
                            booster= 'gbtree',
                            learning_rate = learningRate,
                            base_score= 0.5,
                            alpha= 10,
                            n_estimators= 500,
                            seed= 0,
                            silent= 1,
                            eta= 0.3,
                            gamma= 0,
                            min_child_weight= 1,
                            max_delta_step= 0,
                            subsample= 1,
                            colsample_bytree= 1,
                            reg_lambda= 0,
                            scale_pos_weight= 1,
                            missing= None,
                            verbosity= 1,
                            nthread = 8)
    #model = GridSearchCV( xgbModel, cv=10 )
    xgbModel.fit( xtr, ytr, early_stopping_rounds=20, eval_set=eval_set, eval_metric="logloss", verbose=True )
    #print ( model.best_params_ )
    oof[val_idx] = xgbModel.predict( train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    '''
    pred=xgbModel.predict( xvl )
    print('accuracy_score',accuracy_score( yvl, pred ))
    print("CV score: {:<8.5f}".format(roc_auc_score(yvl, pred)))
    idx += 1
    '''
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = xgbModel.predict( xvl)
sub_df.to_csv("submission_test.csv", index=False)


1 of kfold 10


XGBoostError: b'[16:20:06] src/objective/regression_obj.cc:44: Check failed: preds->Size() == info.labels_.size() (179999 vs. 20001) labels are not correctly providedpreds.size=179999, label.size=20001'

In [87]:
idx         = 1
kf          = StratifiedKFold( n_splits = 10, random_state = 44000, shuffle = False )
eval_set    = [(X_train, y_train)]
learningRate = 0.05
xgbModel = xgb.XGBClassifier(   max_depth = 5,
                                objective = 'binary:logistic',
                                booster= 'gbtree',
                                learning_rate = learningRate,
                                base_score= 0.5,
                                alpha= 10,
                                n_estimators= 500,
                                seed= 0,
                                silent= 1,
                                eta= 0.05,
                                gamma= 0,
                                min_child_weight= 1,
                                max_delta_step= 0,
                                subsample= 1,
                                colsample_bytree= 1,
                                reg_lambda= 0,
                                scale_pos_weight= 1,
                                missing= None,
                                verbosity= 1,
                                nthread = 8)
xgb_param = xgbModel.get_xgb_params()
xgtrain = xgb.DMatrix( X_train, label=train_df['target'])
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv( xgb_param, xgtrain, num_boost_round=100, nfold=5, metrics=['auc'], early_stopping_rounds=10, stratified=True, seed=1301)
print('Best number of trees = {}'.format(cvresult.shape[0]))
xgbModel.set_params(n_estimators=cvresult.shape[0])

for train_index, val_index in kf.split( X_train.values, y_train.values ):
    print('\n{} of kfold {}'.format( idx, kf.n_splits ))
    xtr, xvl = X_train.loc[ train_index ], y_train.loc[ val_index ]
    ytr, yvl = X_train.loc[ train_index ], y_train.loc[ val_index ]
    xgbModel.train( xtr, ytr, early_stopping_rounds=20, eval_set=eval_set, eval_metric="logloss", verbose=True )
    pred = xgbModel.predict( xvl )
    print('accuracy_score',accuracy_score( yvl, pred ))
    print("CV score: {:<8.5f}".format(roc_auc_score(yvl, pred)))
    idx += 1

MemoryError: 